# Image Superresolution

The following notebook shows how to obtain a superresolution image from a set of images of the same object. 

It is very important, that the images are not perfectly aligned, so that each pixel of the cameras sensor captures a slight different part of the subject. The below presented method can be recreated with Adobe Photoshop and the whole idea is inspired by the following video: 
https://www.youtube.com/watch?v=2QW9vcnb9c0&t=207s

To create a superresolution image, several steps are necessary:
1.   Upscaling the images to the desired, new resolution
2.   Aligning the images (we do that by matching SIFT feature points)
3.   Computing the average out of the aligned images 

Ideally, computing the mean has the effect of **increasing sharpness**, **removing artifacts** as well as **reducing noise**. Additionally, moving subjects (e.g. water) will be blurred.



In [ ]:
import cv2
import os
from src.helper.plot import *
from src.helper.superresolution_pipeline import *

### Define the amount of upscaling

We want to have a 5-time bigger image than before

In [ ]:
resize_scale = 5

### Read the images

In the repositories `data/` folder are seven unaligned images

In [ ]:
filenames = ['img01.jpg', 'img02.jpg', 'img03.jpg', 'img04.jpg', 'img05.jpg', 'img06.jpg', 'img07.jpg']
for i in range(0, len(filenames)): 
  filenames[i] = os.path.join('..', 'data', filenames[i])

imgs = read_images(filenames)
plot_images(imgs, title='Images')

### Image upscaling

Just simple upscaling of each image (with bilinear interpolation)

In [ ]:
for i in range(0, len(imgs)):
  imgs[i] = resize_img(imgs[i], scale_factor=resize_scale)

### Feature extraction and matching

Extract SIFT feature points and match the feature points in both images

In [ ]:
num_keypoints = 50

imgs_gray = convert_to_grayscale(imgs)

keypoints, descriptors = extract_features(imgs_gray)

matches = match_descriptors_to_first_image(descriptors, num_keypoints)

plot_matches(imgs[0], keypoints[0], imgs[1], keypoints[1], matches[0], fig_size=[15, 12])

### Image Alignment

Use matched feature points to compute the [homohraphy matrix](https://en.wikipedia.org/wiki/Homography_(computer_vision)) to align the images

In [ ]:
aligned_images = warp_images_to_first_image(matches, keypoints, imgs)

### Superresolution Image

Compute the median of all aligned images to obtain a superresolution image

In [ ]:
superres_img = compute_average_image(aligned_images, algorithm='mean')
cv2.imwrite('superres.jpg', np.asarray(superres_img))

### Plot crop of superresolution image

With this we can see that the superresolution image has a lot less upscaling artifacts than a simply upscaled version of the image

In [ ]:
superres_img = cv2.imread('superres.jpg')
y_offset = 2800
x_offset = 2000
crop_size = 800
plot_image(superres_img[y_offset:y_offset+crop_size, x_offset:x_offset+crop_size], title='Crop of Superres Image', fig_size=[10, 10])
low_res = imgs[0]
plot_image(low_res[y_offset:y_offset+crop_size, x_offset:x_offset+crop_size], title='Crop of Upscaled Image', fig_size=[10, 10])